# Main Library

In [18]:
# Reading Data
import os, yaml
import numpy as np
import string
import nltk

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import tensorflow.keras as k
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

# Reading Data

In [2]:
path_folder = r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V2\data_V2"
files = os.listdir(path_folder)

questions = []
answers = []

for folder in files:
    data = open(path_folder + os.sep + folder, "rb")
    docs = yaml.safe_load(data)
    
    conversations = docs["conversations"]
    for i in conversations:
        if len(i) > 2:
            questions.append(i[0])
            ans = i[1:]
            responses = ""
            for res in ans: responses += " " + res  
            answers.append(responses)
            
        elif len(i) == 2:
            questions.append(i[0])
            answers.append(i[1])
        

# Tokenize Questions

In [3]:
encoder_questions = questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(encoder_questions)
tkz_encoder_questions = tokenizer.texts_to_sequences(encoder_questions)

lon_qus = 0
for qus in tkz_encoder_questions:
    if len(qus) > lon_qus: lon_qus = len(qus)   
print("The Longest Question is ", {lon_qus})

padded_tkz_encoder_question = pad_sequences(tkz_encoder_questions, maxlen=lon_qus, padding="post")
encoder_questions = np.array(padded_tkz_encoder_question)
print("The Encoder Shape is , " ,{encoder_questions.shape})

question_word_dict = tokenizer.word_index
question_tokens = len(question_word_dict)+1
print("The number of tokenize is , " ,{question_tokens})

The Longest Question is  {22}
The Encoder Shape is ,  {(566, 22)}
The number of tokenize is ,  {518}


# Tokenize Answers

In [4]:
decoder_answers = []
for ans in answers: decoder_answers.append("<START>" + ans + "<END>")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(decoder_answers)
tkz_decoder_answers = tokenizer.texts_to_sequences(decoder_answers)

lon_ans = 0
for qus in tkz_decoder_answers:
    if len(qus) > lon_ans: lon_ans = len(qus)   
print("The Longest Question is ", {lon_ans})

padded_tkz_decoder_answers = pad_sequences(tkz_decoder_answers, maxlen=lon_ans, padding="post")
padded_decoder = np.array(padded_tkz_decoder_answers)
print("The Decoder Shape is , " ,{padded_decoder.shape})

answer_word_dict = tokenizer.word_index
answer_token = len(answer_word_dict)+1
print("The number of tokenize is , " ,{answer_token})

The Longest Question is  {74}
The Decoder Shape is ,  {(566, 74)}
The number of tokenize is ,  {1692}


In [5]:
decoder_target_data = []

for token in tkz_decoder_answers:
    decoder_target_data.append(token[1:])
    
temp = pad_sequences(decoder_target_data, maxlen=lon_ans, padding="post")

onehot_padded_answers = to_categorical(temp, answer_token)
decoder_target_data = np.array(onehot_padded_answers)

# Build Model Encoder & Decoder

In [7]:
encoder_inputs = Input(shape=(None,))
encoder_embeding = Embedding(question_tokens, 200, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(200, return_state=True)(encoder_embeding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embeding = Embedding(answer_token, 200, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(200, return_state=True, return_sequences=True)
decoder_output, _, _ = decoder_lstm(decoder_embeding, initial_state=encoder_states)
decoder_dense = Dense(answer_token, activation="softmax")
output = decoder_dense(decoder_output)

model = k.models.Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer=k.optimizers.RMSprop(), loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 200)    103600      ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 200)    338400      ['input_4[0][0]']                
                                                                                              

In [10]:
history = model.fit([encoder_questions, padded_decoder], decoder_target_data, epochs=500, batch_size=300)

Epoch 1/500
2/2 [==============================] - 8s 98ms/step - loss: 1.4408 - accuracy: 0.0667
Epoch 2/500
2/2 [==============================] - 0s 97ms/step - loss: 1.4246 - accuracy: 0.1454
Epoch 3/500
2/2 [==============================] - 0s 98ms/step - loss: 1.2737 - accuracy: 0.0841
Epoch 4/500
2/2 [==============================] - 0s 99ms/step - loss: 1.1545 - accuracy: 0.0708
Epoch 5/500
2/2 [==============================] - 0s 91ms/step - loss: 1.1270 - accuracy: 0.1005
Epoch 6/500
2/2 [==============================] - 0s 92ms/step - loss: 1.1185 - accuracy: 0.0769
Epoch 7/500
2/2 [==============================] - 0s 88ms/step - loss: 1.1131 - accuracy: 0.0744
Epoch 8/500
2/2 [==============================] - 0s 94ms/step - loss: 1.1095 - accuracy: 0.1067
Epoch 9/500
2/2 [==============================] - 0s 97ms/step - loss: 1.1065 - accuracy: 0.1257
Epoch 10/500
2/2 [==============================] - 0s 86ms/step - loss: 1.1027 - accuracy: 0.1274
Epoch 11/500
2/2 [=

2/2 [==============================] - 0s 86ms/step - loss: 0.4738 - accuracy: 0.4896
Epoch 166/500
2/2 [==============================] - 0s 85ms/step - loss: 0.4706 - accuracy: 0.4923
Epoch 167/500
2/2 [==============================] - 0s 86ms/step - loss: 0.4684 - accuracy: 0.4907
Epoch 168/500
2/2 [==============================] - 0s 85ms/step - loss: 0.4643 - accuracy: 0.4974
Epoch 169/500
2/2 [==============================] - 0s 86ms/step - loss: 0.4613 - accuracy: 0.5028
Epoch 170/500
2/2 [==============================] - 0s 84ms/step - loss: 0.4594 - accuracy: 0.5024
Epoch 171/500
2/2 [==============================] - 0s 85ms/step - loss: 0.4591 - accuracy: 0.5042
Epoch 172/500
2/2 [==============================] - 0s 84ms/step - loss: 0.4517 - accuracy: 0.5183
Epoch 173/500
2/2 [==============================] - 0s 86ms/step - loss: 0.4480 - accuracy: 0.5209
Epoch 174/500
2/2 [==============================] - 0s 85ms/step - loss: 0.4470 - accuracy: 0.5180
Epoch 175/500


2/2 [==============================] - 0s 85ms/step - loss: 0.2458 - accuracy: 0.7335
Epoch 248/500
2/2 [==============================] - 0s 84ms/step - loss: 0.2397 - accuracy: 0.7447
Epoch 249/500
2/2 [==============================] - 0s 85ms/step - loss: 0.2370 - accuracy: 0.7497
Epoch 250/500
2/2 [==============================] - 0s 84ms/step - loss: 0.2347 - accuracy: 0.7527
Epoch 251/500
2/2 [==============================] - 0s 87ms/step - loss: 0.2337 - accuracy: 0.7537
Epoch 252/500
2/2 [==============================] - 0s 84ms/step - loss: 0.2330 - accuracy: 0.7518
Epoch 253/500
2/2 [==============================] - 0s 84ms/step - loss: 0.2312 - accuracy: 0.7550
Epoch 254/500
2/2 [==============================] - 0s 83ms/step - loss: 0.2273 - accuracy: 0.7612
Epoch 255/500
2/2 [==============================] - 0s 85ms/step - loss: 0.2261 - accuracy: 0.7640
Epoch 256/500
2/2 [==============================] - 0s 86ms/step - loss: 0.2274 - accuracy: 0.7623
Epoch 257/500


2/2 [==============================] - 0s 86ms/step - loss: 0.1082 - accuracy: 0.9013
Epoch 330/500
2/2 [==============================] - 0s 83ms/step - loss: 0.1073 - accuracy: 0.9011
Epoch 331/500
2/2 [==============================] - 0s 85ms/step - loss: 0.1048 - accuracy: 0.9024
Epoch 332/500
2/2 [==============================] - 0s 84ms/step - loss: 0.1013 - accuracy: 0.9082
Epoch 333/500
2/2 [==============================] - 0s 85ms/step - loss: 0.1007 - accuracy: 0.9096
Epoch 334/500
2/2 [==============================] - 0s 85ms/step - loss: 0.1003 - accuracy: 0.9094
Epoch 335/500
2/2 [==============================] - 0s 83ms/step - loss: 0.0989 - accuracy: 0.9096
Epoch 336/500
2/2 [==============================] - 0s 88ms/step - loss: 0.0995 - accuracy: 0.9085
Epoch 337/500
2/2 [==============================] - 0s 87ms/step - loss: 0.1001 - accuracy: 0.9094
Epoch 338/500
2/2 [==============================] - 0s 84ms/step - loss: 0.0954 - accuracy: 0.9142
Epoch 339/500


2/2 [==============================] - 0s 85ms/step - loss: 0.0417 - accuracy: 0.9648
Epoch 412/500
2/2 [==============================] - 0s 84ms/step - loss: 0.0411 - accuracy: 0.9649
Epoch 413/500
2/2 [==============================] - 0s 72ms/step - loss: 0.0400 - accuracy: 0.9650
Epoch 414/500
2/2 [==============================] - 0s 84ms/step - loss: 0.0400 - accuracy: 0.9663
Epoch 415/500
2/2 [==============================] - 0s 85ms/step - loss: 0.0440 - accuracy: 0.9590
Epoch 416/500
2/2 [==============================] - 0s 86ms/step - loss: 0.0402 - accuracy: 0.9649
Epoch 417/500
2/2 [==============================] - 0s 86ms/step - loss: 0.0374 - accuracy: 0.9679
Epoch 418/500
2/2 [==============================] - 0s 86ms/step - loss: 0.0365 - accuracy: 0.9681
Epoch 419/500
2/2 [==============================] - 0s 85ms/step - loss: 0.0361 - accuracy: 0.9690
Epoch 420/500
2/2 [==============================] - 0s 84ms/step - loss: 0.0359 - accuracy: 0.9677
Epoch 421/500


2/2 [==============================] - 0s 82ms/step - loss: 0.0190 - accuracy: 0.9753
Epoch 494/500
2/2 [==============================] - 0s 78ms/step - loss: 0.0189 - accuracy: 0.9753
Epoch 495/500
2/2 [==============================] - 0s 80ms/step - loss: 0.0186 - accuracy: 0.9750
Epoch 496/500
2/2 [==============================] - 0s 82ms/step - loss: 0.0184 - accuracy: 0.9761
Epoch 497/500
2/2 [==============================] - 0s 79ms/step - loss: 0.0186 - accuracy: 0.9743
Epoch 498/500
2/2 [==============================] - 0s 79ms/step - loss: 0.0190 - accuracy: 0.9744
Epoch 499/500
2/2 [==============================] - 0s 79ms/step - loss: 0.0188 - accuracy: 0.9753
Epoch 500/500
2/2 [==============================] - 0s 80ms/step - loss: 0.0194 - accuracy: 0.9744


In [15]:
def create_encoder_decoder_models():
    ## Encoder Model
    encoder_model = k.models.Model(encoder_inputs, encoder_states)
    
    ## Decoder Model
    
    decoder_input_state_h = Input(shape=(200, ))
    decoder_input_state_c = Input(shape=(200, ))
    decoder_input_states = [decoder_input_state_h, decoder_input_state_c]
    
    decoder_output, state_h, state_c = decoder_lstm(decoder_embeding, initial_state=decoder_input_states)
    
    decoder_output_states = [state_h, state_c]
    decoder_output = decoder_dense(decoder_output)
    
    decoder_model = k.models.Model([decoder_inputs] + decoder_input_states,
                                   [decoder_output] + decoder_output_states)
    
    return encoder_model, decoder_model

In [16]:
enc_model, dec_model = create_encoder_decoder_models()

In [17]:
enc_model.save(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V2\enc_model.h5")
dec_model.save(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V2\dec_model.h5")

In [51]:
def questions_to_tokens(question):

    question = question.translate(str.maketrans('', '', string.punctuation))
    words = question.lower().split()
    tokens = []

    for word in words:
        try:
            tokens.append(question_word_dict[word])
        except:
            print("THe Question is not Recognize")
            run(enc_model, dec_model)

    return pad_sequences([tokens], maxlen=lon_qus, padding="post")

# Load Encoder & Decoder Model

In [33]:
enc_model = k.models.load_model(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V2\enc_model.h5", compile=False)
dec_model = k.models.load_model(r"D:\Courses language programming\9_Big Projects\6 - Building ChatBot\ChatBot - V2\dec_model.h5", compile=False)

In [52]:
def run(enc_model, dec_model):
    while True:
        states_value = enc_model.predict(questions_to_tokens(input("Enter Your Question: ")))
        token = np.zeros((1, 1))

        token[0, 0] = answer_word_dict["start"]
        stop_condition = False
        chatBot_answer = ""

        while not stop_condition:
            dec_output, h, c=  dec_model.predict([token]+states_value)
            index = np.argmax(dec_output[0, 0, :])
            word = list(answer_word_dict.keys())[index-1]
            chatBot_answer += " " + word

            if word == "end" or len(chatBot_answer.split()) > lon_ans : stop_condition = True
            token[0, 0] = index
            states_value = [h, c]

        print(chatBot_answer[:-3])

In [55]:
# run(enc_model, dec_model)